In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
import numpy as np
from datetime import datetime, timedelta

<built-in method __dir__ of sqlite3.Connection object at 0x7f353e178d40>


In [2]:
top_params, df = utils.prepare_best_params_index_retrain()
df

,data,symbol,interval,target_margin,p_ema,min_rsi,max_rsi,amount_invested,stop_loss_multiplier,pnl,arguments
101000,2023-11-17 01:41:37,ADAUSDT,15m,5.0,75,36,78,100.0,4,128.32,"['-batch-training', '-calc-rsi', '-verbose', '..."
101001,2023-11-17 01:37:37,ADAUSDT,15m,1.0,225,36,76,100.0,4,128.33,"['-batch-training', '-calc-rsi', '-verbose', '..."
101002,2023-11-17 01:41:40,ADAUSDT,15m,5.0,100,34,80,100.0,4,128.34,"['-batch-training', '-calc-rsi', '-verbose', '..."
101003,2023-11-17 01:41:32,ADAUSDT,15m,5.0,50,34,80,100.0,4,128.34,"['-batch-training', '-calc-rsi', '-verbose', '..."
101004,2023-11-17 01:41:44,ADAUSDT,15m,5.0,125,34,80,100.0,4,128.34,"['-batch-training', '-calc-rsi', '-verbose', '..."
...,...,...,...,...,...,...,...,...,...,...,...
85995,2023-11-17 04:02:25,XRPUSDT,5m,4.5,150,30,80,100.0,6,230.97,"['-batch-training', '-calc-rsi', '-verbose', '..."
85996,2023-11-17 04:02:48,XRPUSDT,5m,4.5,200,30,80,100.0,7,231.28,"['-batch-training', '-calc-rsi', '-verbose', '..."
85997,2023-11-17 04:02:48,XRPUSDT,5m,4.5,200,30,80,100.0,5,231.28,"['-batch-training', '-calc-rsi', '-verbose', '..."
85998,2023-11-17 04:02:49,XRPUSDT,5m,4.5,200,30,80,100.0,6,231.28,"['-batch-training', '-calc-rsi', '-verbose', '..."


In [ ]:
df_ix_symbol = df[['symbol', 'interval']].drop_duplicates(ignore_index=True).to_dict(orient='records')
df_ix_symbol

In [9]:
import pytz
datetime.now(pytz.utc) - timedelta(days=myenv.days_to_validate_train)

datetime.datetime(2023, 8, 28, 22, 10, 54, 766734, tzinfo=<UTC>)

In [4]:
for i in ['1m', '5m', '15m', '30m', '1h']:
	_, a = utils.get_start_date_for_interval(i)
	print(f'Interval: {i} - Date: {a}') #to_datetime(a)

Interval: 1m - Date: 2023-08-27 21:48:21.127900+00:00
Interval: 5m - Date: 2023-08-26 21:48:21.127990+00:00
Interval: 15m - Date: 2023-08-22 21:48:21.128009+00:00
Interval: 30m - Date: 2023-08-16 21:48:21.128029+00:00
Interval: 1h - Date: 2023-08-04 21:48:21.128040+00:00


In [3]:
def to_datetime(i: int):
	return datetime.fromtimestamp(int(i / 1000) )

def valida(df: pd.DataFrame, interval):
	df.index = df['open_time']
	df.sort_index(inplace=True)
	symbol = df.tail(1)['symbol'].values[0]
	min = to_datetime(df.index.min())
	max = to_datetime(df.index.max())
	print(f'symbol: {symbol} - interval: {interval} - min: {min} - max: {max}')

	result = True
	aux_date = min
	line = 2
	line_erros = 0
	for _, row in df.iterrows():
		opentime = to_datetime(row['open_time'])
		if opentime != aux_date:
			print(f'DIFF::Index[{row["open_time"]}] - Line: {line} - {row["symbol"]}_{interval}: open_time: {opentime} - validate_time: {aux_date}')
			result = False
			aux_date = opentime
			line_erros += 1
		aux_date = aux_date + utils.increment_time(interval)
		line += 1
		
	print(f'Total erros: {line_erros}')
	return result
	

In [4]:
symbol_list = utils.get_symbol_list()
for symbol in symbol_list:
	for interval in myenv.interval_list:
		df = utils.get_database(symbol, interval, columns=myenv.all_cols, parse_dates=False)
		if df.shape[0] > 0:
			valida(df, interval)


symbol: BTCUSDT - interval: 1m - min: 2023-07-31 21:00:00 - max: 2023-11-25 13:35:00
Total erros: 0
symbol: BTCUSDT - interval: 5m - min: 2023-02-28 12:55:00 - max: 2023-11-25 13:35:00
DIFF::Index[1679666400000] - Line: 6875 - BTCUSDT_5m: open_time: 2023-03-24 11:00:00 - validate_time: 2023-03-24 09:40:00
Total erros: 1
symbol: BTCUSDT - interval: 15m - min: 2021-11-25 13:00:00 - max: 2023-11-25 13:30:00
DIFF::Index[1679666400000] - Line: 46453 - BTCUSDT_15m: open_time: 2023-03-24 11:00:00 - validate_time: 2023-03-24 09:45:00
Total erros: 1
symbol: BTCUSDT - interval: 30m - min: 2020-11-25 13:00:00 - max: 2023-11-25 13:30:00
DIFF::Index[1606719600000] - Line: 222 - BTCUSDT_30m: open_time: 2020-11-30 04:00:00 - validate_time: 2020-11-30 03:00:00
DIFF::Index[1608573600000] - Line: 1244 - BTCUSDT_30m: open_time: 2020-12-21 15:00:00 - validate_time: 2020-12-21 11:00:00
DIFF::Index[1608865200000] - Line: 1404 - BTCUSDT_30m: open_time: 2020-12-25 00:00:00 - validate_time: 2020-12-24 23:00:00